In [92]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
M06 – Demonstrasi penyelesaian sistem persamaan non-linear:

  f1(x,y) = x^2 + x*y - 10 = 0
  f2(x,y) = y + 3*x*y^2 - 57 = 0

Metode:
- Iterasi Titik Tetap (Jacobi dan Seidel) dengan g1A & g2A
- Newton–Raphson (2D)
- Secant (skalar) pada φ(x) setelah mengeliminasi y dari f1

Toleransi: 1e-6
Tebakan awal: x0=1.5, y0=3.5 ; Secant: x0=1.5, x1=2.5
"""

import math
import numpy as np

TOL = 1e-6
MAXIT = 50  # Batasi iterasi untuk metode divergen


In [93]:
# -------------------------------
# Definisi sistem & turunannya
# -------------------------------
def f1(x, y):
    return x*x + x*y - 10.0

def f2(x, y):
    return y + 3.0*x*(y*y) - 57.0

def J(x, y):
    """Jacobian untuk Newton-Raphson 2D."""
    return np.array([
        [2.0*x + y,    x],
        [3.0*y*y,  1.0 + 6.0*x*y]
    ], dtype=float)


In [94]:
# -------------------------------------
# Fungsi iterasi g1A & g2A (halaman 5 PDF)
# -------------------------------------
def g1A(x, y):
    # x_{r+1} = (10 - x_r^2) / y_r
    if y == 0:
        return float('nan')
    return (10.0 - x*x) / y

def g2A(x, y):
    # y_{r+1} = 57 - 3*x_r*y_r^2
    return 57.0 - 3.0*x*(y*y)


In [95]:
# -------------------------------
# Iterasi Titik Tetap
# -------------------------------
def fixed_point_jacobi(x0, y0, tol=TOL, maxit=MAXIT):
    """Jacobi: x_{k+1} = g1A(x_k, y_k), y_{k+1} = g2A(x_k, y_k)"""
    x, y = x0, y0
    hist = [(0, x, y, 0.0, 0.0)]
    for k in range(1, maxit+1):
        xn = g1A(x, y)
        yn = g2A(x, y)
        dx, dy = abs(xn - x), abs(yn - y)
        hist.append((k, xn, yn, dx, dy))
        x, y = xn, yn
        if dx < tol and dy < tol:
            break
        if not (math.isfinite(x) and math.isfinite(y)):
            break
    return hist


def fixed_point_seidel(x0, y0, tol=TOL, maxit=MAXIT):
    """Seidel: x_{k+1} = g1A(x_k, y_k), y_{k+1} = g2A(x_{k+1}, y_k)"""
    x, y = x0, y0
    hist = [(0, x, y, 0.0, 0.0)]
    for k in range(1, maxit+1):
        xn = g1A(x, y)
        yn = g2A(xn, y)  # gunakan xn baru
        dx, dy = abs(xn - x), abs(yn - y)
        hist.append((k, xn, yn, dx, dy))
        x, y = xn, yn
        if dx < tol and dy < tol:
            break
        if not (math.isfinite(x) and math.isfinite(y)):
            break
    return hist


In [96]:
# -------------------------------
# Newton–Raphson 2D
# -------------------------------
def newton_raphson(x0, y0, tol=TOL, maxit=MAXIT):
    x, y = x0, y0
    hist = [(0, x, y, 0.0, 0.0, f1(x,y), f2(x,y))]
    for k in range(1, maxit+1):
        F = np.array([f1(x,y), f2(x,y)], dtype=float)
        try:
            delta = np.linalg.solve(J(x,y), -F)
        except np.linalg.LinAlgError:
            break
        xn, yn = x + float(delta[0]), y + float(delta[1])
        dx, dy = abs(xn - x), abs(yn - y)
        hist.append((k, xn, yn, dx, dy, f1(xn,yn), f2(xn,yn)))
        x, y = xn, yn
        if max(dx, dy) < tol:
            break
    return hist


In [97]:
# -------------------------------
# Secant (skalar) pada φ(x)
# -------------------------------
def y_from_x(x):
    # Dari f1=0: y = 10/x - x
    if x == 0:
        return float('nan')
    return 10.0/x - x

def phi(x):
    return f2(x, y_from_x(x))

def secant_scalar(x0, x1, tol=TOL, maxit=MAXIT):
    hist = [(0, x0, y_from_x(x0), float('nan'))]
    f0 = phi(x0)
    hist.append((1, x1, y_from_x(x1), abs(x1-x0)))
    f1v = phi(x1)
    for k in range(2, maxit+1):
        denom = (f1v - f0)
        if denom == 0 or not np.isfinite(denom):
            break
        x2 = x1 - f1v*(x1 - x0)/denom
        err = abs(x2 - x1)
        hist.append((k, x2, y_from_x(x2), err))
        if err < tol:
            break
        x0, f0, x1, f1v = x1, f1v, x2, phi(x2)
    return hist


In [98]:
# -------------------------------
# Utilitas cetak & analisis
# -------------------------------
def print_table(rows, header, floatfmt="{:.6f}", max_rows=30):
    print(header)
    print("-"*len(header))
    cols = len(rows[0])

    if not rows or not math.isfinite(rows[-1][1]):
        print("Solusi divergen atau menghasilkan nilai tidak valid.\n")
        return

    if cols == 5:
        print(f"{'iter':>4} {'x':>15} {'y':>15} {'|Δx|':>15} {'|Δy|':>15}")
    elif cols == 7:
        print(f"{'iter':>4} {'x':>15} {'y':>15} {'|Δx|':>15} {'|Δy|':>15} {'f1':>15} {'f2':>15}")
    elif cols == 4:
        print(f"{'iter':>4} {'x':>15} {'y (from f1)':>15} {'|Δx|':>15}")

    for r in rows[:max_rows]:
        if cols == 5:
            k,x,y,dx,dy = r
            print(f"{k:4d} {floatfmt.format(x):>15} {floatfmt.format(y):>15} "
                  f"{floatfmt.format(dx):>15} {floatfmt.format(dy):>15}")
        elif cols == 7:
            k,x,y,dx,dy,v1,v2 = r
            print(f"{k:4d} {floatfmt.format(x):>15} {floatfmt.format(y):>15} "
                  f"{floatfmt.format(dx):>15} {floatfmt.format(dy):>15} "
                  f"{floatfmt.format(v1):>15} {floatfmt.format(v2):>15}")
        elif cols == 4:
            k,x,y,dx = r
            print(f"{k:4d} {floatfmt.format(x):>15} {floatfmt.format(y):>15} "
                  f"{floatfmt.format(dx):>15}")
    print()

def estimate_order(errors):
    """Estimasi orde p dari tiga galat terakhir yang >0."""
    e = [err for err in errors if err > 1e-12]
    if len(e) < 3:
        return float("nan")
    ekm1, ek, ekp1 = e[-3], e[-2], e[-1]
    try:
        order = math.log(ekp1/ek)/math.log(ek/ekm1)
        return order
    except (ValueError, ZeroDivisionError):
        return float("nan")


In [99]:
# -------------------------------
# Main demo
# -------------------------------
def main():
    x0, y0 = 1.5, 3.5
    sx0, sx1 = 1.5, 2.5

    # Jalankan metode
    jacobi = fixed_point_jacobi(x0, y0, TOL, MAXIT)
    seidel = fixed_point_seidel(x0, y0, TOL, MAXIT)
    newton = newton_raphson(x0, y0, TOL, MAXIT)
    secant = secant_scalar(sx0, sx1, TOL, MAXIT)

    # Cetak tabel hasil
    print_table(jacobi, "Iterasi Titik Tetap – Jacobi (g1A, g2A)", max_rows=10)
    print_table(seidel, "Iterasi Titik Tetap – Seidel (g1A, g2A)", max_rows=10)
    print_table(newton, "Newton–Raphson (2D)", max_rows=10)
    print_table(secant, "Secant (skalar) pada φ(x)", max_rows=10)

    # Analisis konvergensi
    print("--- Analisis Konvergensi dan Kecepatan ---")
    xn, yn = newton[-1][1], newton[-1][2]
    print(f"Akar ditemukan (via Newton): x* ≈ {xn:.6f}, y* ≈ {yn:.6f}\n")

    # Ringkasan iterasi
    print("Jumlah Iterasi untuk Mencapai Toleransi:")
    print(f"- Jacobi (g1A, g2A): {'Divergen' if not math.isfinite(jacobi[-1][1]) else jacobi[-1][0]}")
    print(f"- Seidel (g1A, g2A): {'Divergen' if not math.isfinite(seidel[-1][1]) else seidel[-1][0]}")
    print(f"- Newton-Raphson   : {newton[-1][0]}")
    print(f"- Secant (skalar)  : {secant[-1][0]}\n")

    # Estimasi orde
    def vec_err(seq, x_true, y_true):
        return [math.hypot(r[1]-x_true, r[2]-y_true) for r in seq]
    def scalar_err(seq, x_true):
        return [abs(r[1]-x_true) for r in seq]

    p_newton = estimate_order(vec_err(newton, xn, yn))
    p_secant = estimate_order(scalar_err(secant, xn))
    print("Perkiraan Orde Konvergensi:")
    print(f"- Newton-Raphson: {p_newton:.3f} (teoritis ≈ 2)")
    print(f"- Secant        : {p_secant:.3f} (teoritis ≈ 1.618)")

# Jalankan
main()


Iterasi Titik Tetap – Jacobi (g1A, g2A)
---------------------------------------
iter               x               y            |Δx|            |Δy|
   0        1.500000        3.500000        0.000000        0.000000
   1        2.214286        1.875000        0.714286        1.625000
   2        2.718367       33.646205        0.504082       31.771205
   3        0.077586    -9175.123002        2.640781     9208.769208
   4       -0.001089 -19594215.888012        0.078675 19585040.765009
   5       -0.000001 1254595224668.735352        0.001089 1254614818884.623291
   6        0.000000 2409908617942726144.000000        0.000001 2409907363347501568.000000
   7        0.000000 -138873305891179259475525632.000000        0.000000 138873308301087883162288128.000000
   8       -0.000000 -240081241407512004850179587218341888.000000        0.000000 240081241268638717357285316243750912.000000
   9       -0.000000 12451421554176399250472379260600122123340480512.000000        0.000000 124514215